# Analytics usando tipos de dados complexos
<p><img src = https://thumbs.dreamstime.com/b/cores-em-ascens%C3%A3o-superf%C3%ADcie-tridimensional-colorida-de-indiv%C3%ADduos-com-dados-complexos-arco-%C3%ADris-como-malha-arame-preto-ou-241163870.jpg width=500></p>

## Introdução
No módulo anterior, vimos como podemos importar e exportar dados para outras ferramentas analíticas para aproveitar as ferramentas analíticas fora do nosso banco de dados. Muitas vezes é mais fácil analisar números, mas no mundo real, os dados são frequentemente encontrados em outros formatos: palavras, locais, datas e, às vezes, estruturas de dados complexas. Neste módulo, veremos esses outros formatos e veremos como podemos usar esses dados na nossa análise.

Primeiro, veremos dois tipos de coluna comumente encontrados: colunas de data e hora (**datetime**) e colunas de **latitude** e **longitude**. Esses tipos de dados nos darão uma compreensão fundamental de como entender nossos dados de uma perspectiva temporal e geoespacial. Em seguida, veremos tipos de dados complexos, como **arrays** e **JSON**, e aprenderemos como extrair pontos de dados desses tipos de dados complexos. Essas estruturas de dados são frequentemente usadas para dados alternativos ou dados em nível de log, como logs de sites. Por fim, veremos como podemos extrair significado do texto em nosso banco de dados e usar dados de texto para extrair **insights**.
Ao final do módulo, você terá ampliado seus recursos de análise para que possa aproveitar praticamente qualquer tipo de dado disponível para você.
## Tipos de dados de data e hora para análise
Estamos todos familiarizados com datas e horários, mas não pensamos frequentemente em como essas medidas quantitativas são representadas. Sim, eles são representados por números, mas não por um único número. Em vez disso, eles são medidos com um conjunto de números, um para o ano, um para o mês, um para o dia do mês, um para a hora, um para o minuto e assim por diante.

O que podemos não perceber, porém, é que esta é uma representação complexa, composta por vários componentes diferentes. Por exemplo, saber o minuto atual sem saber a hora atual é inútil. Além disso, existem maneiras complexas de interagir com datas e horas, por exemplo, diferentes pontos no tempo podem ser subtraídos um do outro. Além disso, a hora atual pode ser representada de forma diferente dependendo de onde você está no mundo.

Como resultado desses meandros, precisamos ter um cuidado especial ao trabalhar com esse tipo de dados. Na verdade, o *Postgres*, como a maioria dos bancos de dados, oferece tipos de dados especiais que podem representar esses tipos de valores. Começaremos examinando o tipo de data (**date**).

### Começando com o tipo de data
As datas podem ser facilmente representadas usando strings, por exemplo, "1 de janeiro de 2000", que representa claramente uma data específica, mas as datas são uma forma especial de texto, pois representam um valor quantitativo e sequencial. Você pode adicionar uma semana à data atual, por exemplo. Uma determinada data tem muitas propriedades diferentes que você pode querer usar em sua análise, por exemplo, o ano ou o dia da semana que a data representa. Trabalhar com datas também é necessário para a análise de séries temporais, que é um dos tipos mais comuns de análise que surgem.

O padrão SQL inclui um tipo de dados **DATE** e o *PostgreSQL* oferece ótimas funcionalidades para interagir com esse tipo de dados. Primeiro, podemos configurar nosso banco de dados para exibir datas no formato com o qual estamos mais familiarizados. O PostgreSQL usa o parâmetro **DateStyle** para definir essas configurações. Para ver suas configurações atuais, você pode usar o seguinte comando:

`SHOW DateStyle;`

A seguir está a saída da consulta anterior:

In [33]:
# Importando o SqlAlchemy
from sqlalchemy import create_engine

cnxn_string = ("postgresql+psycopg2://{username}:{pswd}"
               "@{host}:{port}/{database}")

# Instanciando a Função create_engine com a string de conexão
engine = create_engine(cnxn_string.format(
    username="postgres",
    pswd="admin",
    host="127.0.0.1",
    port=5432,
    database="sqlda"))

# Criando a Consulta ou o comando SQL a ser executado
sql = """
		SHOW DateStyle
      """

# Executando o Comando SQL
rs = engine.execute(sql)

for row in rs:
    print(row)

('ISO, DMY',)


O primeiro parâmetro especifica o formato de saída da **International Organization Standardization (ISO)**, que exibe a data como Ano, Mês, Dia e o segundo parâmetro especifica a ordem (por exemplo, Mês, Dia, Ano versus Dia, Mês, Ano) para entrada ou resultado. Você pode configurar a saída para seu banco de dados usando o seguinte comando:

`SET DateStyle='ISO, MDY';`

Por exemplo, se você quiser defini-lo para o formato europeu de Dia, Mês, Ano, defina DateStyle como 'GERMAN, DMY'. Para este módulo, usaremos o formato de exibição ISO (Ano, Mês, Dia) e o formato de entrada Mês, Dia, Ano. Você pode configurar esse formato usando o comando anterior.

In [34]:
sql = """
	  SET DateStyle='GERMAN, DMY';
      """

# Executando o Comando SQL
rs = engine.execute(sql)

for row in rs:
	print(row)

ResourceClosedError: This result object does not return rows. It has been closed automatically.